In [1]:
import os
import shutil
import random

# Set the paths to your dataset and the train/test folders
data_dir = "/kaggle/input/covid-19/Covid19"
train_dir = "/kaggle/working/train"
test_dir = "/kaggle/working/test"

# Create the train and test folders if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create subfolders for the classes (Normal and Covid-19)
os.makedirs(os.path.join(train_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "COVID"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "COVID"), exist_ok=True)

# Set the train/test split ratio (e.g., 0.8 for 80% train, 0.2 for 20% test)
train_ratio = 0.8

# Loop through the classes
for class_name in ["Normal", "COVID"]:
    class_dir = os.path.join(data_dir, class_name)
    files = os.listdir(class_dir)
    
    # Shuffle the files randomly
    random.shuffle(files)
    
    # Calculate the number of files for train and test sets
    num_train = int(len(files) * train_ratio)
    
    # Copy the files to the train and test folders
    for i, file in enumerate(files):
        src = os.path.join(class_dir, file)
        if i < num_train:
            dst = os.path.join(train_dir, class_name, file)
        else:
            dst = os.path.join(test_dir, class_name, file)
        shutil.copy(src, dst)

In [2]:
%pip install ipywidgets
%pip install torchvision 
%pip install matplotlib
%pip install -q flwr[simulation] 
%pip install --upgrade flwr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pr

In [3]:
import numpy as np
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import flwr as fl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import IPython.display as display_output
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from collections import OrderedDict
from typing import List, Tuple
from IPython.display import display, clear_output
from IPython.display import display, HTML, clear_output
from flwr.common import Metrics
from flwr.server.strategy.fedprox import FedProx
global num_c, trainloaders, valloaders, testloader, model, classes
num_clients=10

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Required for Kaggle to work with PyTorch

torch.cuda.is_available()  # Check if GPU is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__}"
)
# Dataset paths
dataset_paths = '/kaggle/working/'

datatrain = f'{dataset_paths}/train'
datatest = f'{dataset_paths}/test'

BATCH_SIZE = 32
img_size = (224, 224)

# Download and transform
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = ImageFolder(datatrain, transform=transform)
testset = ImageFolder(datatest, transform=transform)
size_label = widgets.Label()
size_label.value = f'The size of data: {len(trainset) + len(testset)}'
print(size_label)

# Get the classes from the trainset
classes = trainset.classes
num_c = len(classes)

# Split training set into num_clients partitions to simulate the individual dataset
partition_size = len(trainset) // num_clients
remainder = len(trainset) % num_clients
lengths = [partition_size + remainder] + [partition_size] * (num_clients - 1)
datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

# Split each partition into train/val and create DataLoader
trainloaders = []
valloaders = []
for ds in datasets:
    len_val = len(ds) // 10  # 10 % validation set
    len_train = len(ds) - len_val
    lengths = [len_train, len_val]
    ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
    print("hi")
    valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)
    
class Net(nn.Module):
    def __init__(self, num_c):
        super(Net, self).__init__()
        self = self.to(DEVICE)
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 264)
        self.fc2 = nn.Linear(264,num_c)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x
    
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")
def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

penalty_weight = 0.1 
def aggregate_FedProx(metrics: List[Tuple[int, dict]]) -> dict:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate accuracy and examples
    aggregated_metrics = {
        "accuracy": sum(accuracies) / sum(examples),
        "examples": sum(examples)
    }

    # Apply penalty term to the aggregated accuracy
    aggregated_metrics["accuracy_with_penalty"] = aggregated_metrics["accuracy"] + penalty_weight * aggregated_metrics["examples"]

    return aggregated_metrics

def fit_metrics_aggregation_fn(fit_metrics):
    # Aggregate and return custom metric (e.g., weighted average)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in fit_metrics]
    examples = [num_examples for num_examples, _ in fit_metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

class FlowerNumPyClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        
        # Get the accuracy on the training set
        correct, total = 0, 0
        self.net.eval()
        with torch.no_grad():
            for images, labels in self.trainloader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        
        return get_parameters(self.net), len(self.trainloader), {"accuracy": float(accuracy)}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}
    
    
def client_fn(cid) -> FlowerNumPyClient:
    net = Net(num_c).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    flower_numpy_client = FlowerNumPyClient(cid, net, trainloader, valloader)
    return flower_numpy_client.to_client()



def run_simulation(num_round,strategy_dropdown):   
    if strategy_dropdown == "FedProx":
        # Start simulation with FedProx strategy
        strategy = FedProx(
            fraction_fit=1.0,
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            evaluate_metrics_aggregation_fn=aggregate_FedProx,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            proximal_mu=0.1,
        )

    # Update with your specific simulation code
    fl.common.logger.configure(identifier="myFlowerExperiment", filename="log.txt")
    output = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=num_round),
        strategy=strategy,
        client_resources={"num_cpus": 1, "num_gpus": 1},
    )

run_simulation(10,"FedProx")

2024-05-03 14:53:41.554845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 14:53:41.554950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 14:53:41.825258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 14:53:54,292	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `

Training on cuda using PyTorch 2.1.2
Label(value='The size of data: 13808')
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2024-05-03 14:53:57,968	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 8907351244.0, 'GPU': 2.0, 'CPU': 4.0, 'node:__internal_head__': 1.0, 'node:172.19.2.2': 1.0, 'memory': 17814702491.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
(pid=280) 2024-05-03 14:54:02.581873: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN fa

(ClientAppActor pid=280) [Client 5] fit, config: {'proximal_mu': 0.1}
(ClientAppActor pid=278) [Client 7] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(ClientAppActor pid=278) [Client 9] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster]
(ClientAppActor pid=280) [Client 4] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster]
(ClientAppActor pid=280) [Client 6] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 8 clients (out of 10)


...

INFO :      aggregate_evaluate: received 8 results and 0 failures
INFO :      
INFO :      [ROUND 10]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=280) [Client 4] fit, config: {'proximal_mu': 0.1}
(ClientAppActor pid=280) [Client 6] evaluate, config: {} [repeated 7x across cluster]
(ClientAppActor pid=280) [Client 7] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster]
(ClientAppActor pid=280) [Client 5] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster]
(ClientAppActor pid=280) [Client 3] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster]
(ClientAppActor pid=280) [Client 9] fit, config: {'proximal_mu': 0.1} [repeated 2x across cluster]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 8 clients (out of 10)


(ClientAppActor pid=278) [Client 0] evaluate, config: {}
(ClientAppActor pid=278) [Client 1] fit, config: {'proximal_mu': 0.1}


INFO :      aggregate_evaluate: received 8 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 rounds in 573.21s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 0.02303625629706816\n'
INFO :      	 '\tround 2: 0.01580759229307825\n'
INFO :      	 '\tround 3: 0.014050491255792706\n'
INFO :      	 '\tround 4: 0.011700838177718902\n'
INFO :      	 '\tround 5: 0.010561331437731332\n'
INFO :      	 '\tround 6: 0.009585239949890158\n'
INFO :      	 '\tround 7: 0.008922698433426293\n'
INFO :      	 '\tround 8: 0.00814861744557592\n'
INFO :      	 '\tround 9: 0.007580139750445431\n'
INFO :      	 '\tround 10: 0.006937469834123146\n')History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.7409090909090909),
INFO :      	              (2, 0.7852272727272728),
INFO :      	              (3, 0.8374999999999999),
INFO :      	              (4, 0.8511363636363637),
INFO :      	              (5, 0.8647727272727274),
INFO :      